In [1]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

Using TensorFlow backend.


In [2]:
X_train = train["comment_text"].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].str.lower()
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [4]:
vocab = 130000
maxlen = 220
embed_size = 300

In [5]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [6]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [7]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [8]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
def make_model(k=[2,3,5,7], activation='tanh', filters=32, Sdroprate=0.4, droprate=0.0):

    inp = Input(shape=(maxlen, ))
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    x = SpatialDropout1D(Sdroprate)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv = dict()
    maxpool = dict()
    for h in k:
        conv[h] = Conv2D(filters, kernel_size=(h, embed_size), activation=activation)(x)
        maxpool[h] = MaxPool2D(pool_size=(maxlen - h + 1, 1))(conv[h])
        
    y = Concatenate(axis=1)([pool for key,pool in maxpool.items()])
    y = Flatten()(y)
    y = Dropout(droprate)(y)

    outp = Dense(6, activation="sigmoid")(y)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 220)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 220, 300)     39000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 220, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 220, 300, 1)  0           spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [10]:
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=8)

In [11]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [12]:
#RocAuc = RocAucEvaluation(validation_data=(x_val, y_val), interval=1)

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
batch_size=256
epochs = 5
file_path = 'saved_models/weights.best.hdf5'
check_point = ModelCheckpoint(filepath=file_path, verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
roc_auc = RocAuc()

In [13]:
h = model.fit(x_trainS, y_trainS, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
                 callbacks=[check_point, early_stop, roc_auc], verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
 - 698s - loss: 0.0818 - acc: 0.9743 - val_loss: 0.0468 - val_acc: 0.9825

Epoch 00001: val_loss improved from inf to 0.04681, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 1 - score: 0.980844 

Epoch 2/5
 - 693s - loss: 0.0488 - acc: 0.9817 - val_loss: 0.0428 - val_acc: 0.9836

Epoch 00002: val_loss improved from 0.04681 to 0.04283, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 2 - score: 0.986194 

Epoch 3/5
 - 685s - loss: 0.0448 - acc: 0.9828 - val_loss: 0.0414 - val_acc: 0.9839

Epoch 00003: val_loss improved from 0.04283 to 0.04139, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 3 - score: 0.987752 

Epoch 4/5
 - 685s - loss: 0.0427 - acc: 0.9836 - val_loss: 0.0410 - val_acc: 0.9840

Epoch 00004: val_loss improved from 0.04139 to 0.04097, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 4 - score: 0.988421 

Epoch 5/5
 - 685s - loss: 0.0412 - ac

In [14]:
y_pred = model.predict(x_test, batch_size=256)

In [15]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.982330 



In [ ]:
'''submission = pd.read_csv('sample_submission.csv')
y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)

In [16]:
from keras.models import load_model
model_saved = load_model(file_path)

In [17]:
model_saved.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 220)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 220, 300)     39000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 220, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 220, 300, 1)  0           spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [18]:
y_pred = model_saved.predict(x_test, batch_size=256)

In [19]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.981602 

